In [1]:
import os

# Desinstala numpy y cualquier paquete conflictivo
!pip uninstall -y numpy torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -q

# Instala numpy compatible (ANTES de que torch lo cargue)
!pip install numpy==1.24.4

# Mensaje al usuario
print("NumPy 1.24.4 instalado correctamente.")
print("Reiniciando entorno para que los cambios surtan efecto...")

# Reinicia el entorno de ejecución automáticamente (muy importante)
os.kill(os.getpid(), 9)


You can safely remove it manually.


^C


: 

In [ ]:

# Instalación de PyTorch con CUDA 11.8
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# Instalación de dependencias específicas de PyTorch Geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

# Instalación de torch-geometric y librerías útiles
!pip install torch-geometric matplotlib pandas optuna optuna-dashboard

!pip install pyngrok --quiet


In [ ]:
# Eliminar carpeta antigua si existe, para evitar que se quede en  el caché
!rm -rf ia_thermal

# Clonar versión actualizada desde GitHub
!git clone --branch Ernesto https://github.com/dgbarcena/ia_thermal.git

%cd ia_thermal/ernestoemedina3D/SAGE

In [ ]:
# Añadir ruta al sistema para importar módulos locales
import sys
sys.path.append('/content/ia_thermal/ernestoemedina3D/SAGE')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Optuna

In [1]:
import optuna
import torch
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from SAGE_model import GraphSAGE
from train_eval import train, evaluate
from Data_loader import load_dataset

c:\Users\ramse\anaconda3\envs\Spacethermalpy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    dir_path = "/content/drive/MyDrive/ErnestoData"
    db_path = "/content/drive/MyDrive/ErnestoData/Optuna3D/sage_optuna_1.db"
else:
    dir_path = os.getcwd()
    db_path = os.path.join(dir_path, "sage_optuna_1.db")

In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)


Usando dispositivo: cuda


In [4]:
def objective(trial):
    # --- Hiperparámetros a optimizar ---
    num_layers = trial.suggest_int("num_layers", 3, 10)
    hidden_dim = trial.suggest_categorical("hidden_dim", [32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # --- Cargar datos ---
    dataset_path = os.path.join(dir_path, "Datasets3D", "Dataset3D_10000_model.pth")
    
    train_loader, val_loader, test_loader, norm_info = load_dataset(
    path=dataset_path,
    batch_size=batch_size
    )


    # --- Crear modelo ---
    model = GraphSAGE(
        input_dim=2,
        hidden_dim=hidden_dim,
        output_dim=1,
        num_layers=num_layers,
        use_dropout=True,
        dropout_rate=dropout_rate,
        use_batchnorm=True,
        use_residual=True
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # --- Scheduler con ReduceLROnPlateau ---
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=10,
        min_lr=1e-6
    )

    # --- Entrenamiento + Evaluación en validación ---
    for epoch in range(50):
        train(
            model,
            train_loader,
            optimizer,
            device,
            norm_info=norm_info,
            use_physics=True,
            lambda_physics=0.00001,
            use_heater_loss=True,
            lambda_heater=0.1
        )

        val_metrics = evaluate(
            model,
            val_loader,
            device,
            norm_info=norm_info,
            lambda_physics=0.00001,
            use_physics=True,
            lambda_heater=0.1,
            use_heater_loss=True,
            error_threshold_K=3.0
        )

        val_total_loss = val_metrics[-1]
        scheduler.step(val_total_loss)

    return val_total_loss  # Métrica a minimizar

## Estudio

In [5]:
# Crear estudio y ejecutar optimización
study = optuna.create_study(
    direction="minimize",
    study_name="sage_study_1",
    storage=f"sqlite:///{db_path}",
    load_if_exists=True
)

study.optimize(objective, n_trials=100)

[I 2025-06-20 09:31:39,270] Using an existing study with name 'sage_study_1' instead of creating a new one.
c:\Users\ramse\Escritorio\Aero Ordenador\4to Aero\2do Cuatri\Spacethermalpy\ernestoemedina\SAGE\Data_loader.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loa

KeyboardInterrupt: 

In [ ]:

print("Mejor conjunto de hiperparámetros encontrados:")
for key, val in study.best_trial.params.items():
    print(f"{key}: {val}")


In [ ]:

import optuna.visualization.matplotlib as vis

fig1 = vis.plot_optimization_history(study)
fig1.show()

fig2 = vis.plot_param_importances(study)
fig2.show()
